# Data Handling Workflows

Note that Parsl is not effective if multiple CPU cores aren't available because Parsl's ability to execute tasks in parallel is depenedent on the availability multiple cores.

In [1]:
import multiprocessing
print('Cores available: {}'.format(multiprocessing.cpu_count()))

Cores available: 4


### Importing Libraries and Configuration

In [2]:
import numpy as np
import random
import time

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor

config = Config(
    executors=[ThreadPoolExecutor()],
    lazy_errors=True
)
parsl.load(config)

### Map Reduce

A map reduce is a technique to execute multiple parallel jobs on a dataset to reduce the size of the dataset before executing a final function to get the result. A Map reduce is a more complicated version of synchronisation.

Let's consider a simple example where we are given multiple lists and we want to select the lists with the highest standard deviation.

![](./images/map_reduce.png)

In [3]:
# A python app to compute the standard deviation of the inputs
@python_app
def standard_deviation(inputs=[]):
    import numpy as np
    return np.std(inputs)

In [4]:
# A function to construct data that is a list of lists, each each list having 100 random numbers

def make_data():
    lists = []
    
    for _ in range(100):
        new_list = []
        
        for __ in range(100):
            new_list.append(random.random()*100)
            
        lists.append(new_list)
    return lists

our_data = make_data()

### Parallel Execution

In [ ]:
# Computing the standard deviations for each list

start1 = time.time()

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation(inputs=i))

# Finding the maximum standard deviation
standard_deviations = [i.result() for i in standard_deviations]
print('Maximum Standard Deviation: ', max(standard_deviations))

# Finding the list with the maximum standard deviation
maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

end1 = time.time()

# Hashing

We'll be using a simple hash function to store elements in our database. We'll evaluate the hash values in parallel and then store the items in those locations.

In [ ]:
database = [0 for i in range(1000)]  # An empty database

In [4]:
@python_app 
def hash_function(element):
    import hashlib # importing all the libraries inside the remote function
    number = int(hashlib.md5(element).hexdigest()[:8], 16)%1000   # Creating a hash index
    return number

### Parallel Execution

In [ ]:
import random

elements = []

for i in range(100):
    element = '' 
    for _ in range(5):
        element += random.choice('abcdefghijklmopqrstuvwxyz')
    element = element.encode() # Making a 5 letter element
    elements.append(element) # collecting 100 such elements

start1 = time.time()   
hashes = []
for i in elements: # Updating the database for all the elements
    hashes.append(hash_function(i))

hashes = [i.result() for i in hashes]

for i in range(len(elements)):
    database[hashes[i]] = elements[i]

end1 = time.time()

Note that this still doesn't solve the problem of overlap of elements. Chaining is the alternative here but in order to implement chaining, we have to evaluate the results which breaks the parallel thread.

## Montage Mosaic

This Python script has been inspired from the [Montage Wrapper Documentation](https://montage-wrapper.readthedocs.io/en/v0.9.5) and the [tutorial](http://montage.ipac.caltech.edu/docs/first_mosaic_tutorial.html) for the Montage Mosaic.

In [3]:
import montage_wrapper as montage

### First Part

In [8]:
import os
cwd = os.getcwd()
from parsl.data_provider.files import File

In [ ]:
@python_app
def mProject_parsl(inputs=[]):
    import montage_wrapper as montage
    return montage.mProject(inputs[0], inputs[1], inputs[2])

In [10]:
from IPython.utils import io

with io.capture_output() as captured:
    !tar xvf Kimages.tar
    montage.mImgtbl(os.path.join(cwd,'Kimages/'),  File(os.path.join(cwd,'Kimages.tbl')))
    montage.mMakeHdr(File(os.path.join(cwd,'Kimages.tbl')), File(os.path.join(cwd,'Ktemplate.hdr')))
    os.mkdir(os.path.join(cwd,'Kprojdir/'))

In [ ]:
list_of_images = os.listdir(os.path.join(cwd,'Kimages/'))

output = []

for image in list_of_images:
    input_image = File(os.path.join(cwd, 'Kimages/' + image))
    output_image = File(os.path.join(cwd, 'Kprojdir/hdu0_' + image))
    template = File(os.path.join(cwd,'Ktemplate.hdr'))
    
    output.append(mProject_parsl(inputs=[input_image, output_image, template]))
    
output = [i.result() for i in output]
    
# Alternative ProjExec command below:

# montage.mProjExec(File(os.path.join(cwd,'Kimages.tbl')),
                  # File(os.path.join(cwd,'Ktemplate.hdr')),
                  # os.path.join(cwd,'Kprojdir/'),
                  # File(os.path.join(cwd,'stats.tbl')))

In [ ]:
with io.capture_output() as captured2:
    montage.mImgtbl(os.path.join(cwd,'Kprojdir/'), File(os.path.join(cwd,'images.tbl')))
    montage.mAdd( File(os.path.join(cwd,'images.tbl')), 
              File(os.path.join(cwd,'Ktemplate.hdr')), 
              File(os.path.join(cwd,'m17_uncorrected.fits')))
    !mViewer -ct 1 -gray m17_uncorrected.fits -1s max gaussian-log -out m17_uncorrected.png

![](./images/m17_uncorrected.png)

### Second Part

In [22]:
with io.capture_output() as captured:
    montage.mOverlaps(File(os.path.join(cwd,'images.tbl')), File(os.path.join(cwd,'diffs.tbl')))
    os.mkdir(os.path.join(cwd,'diffdir/'))

In [23]:
@python_app
def mDiff_parsl(inputs=[]):
    import montage_wrapper as montage
    return montage.mDiff(inputs[0], inputs[1], inputs[2], inputs[3])

In [ ]:
import pandas as pd
df = pd.read_csv('diffs.tbl', comment='#', delim_whitespace=True).drop(0)
images1 = list(df['|.1'])
images2 = list(df['cntr2'])
outputs = list(df['|.2'])

In [ ]:
outputs_2 = []

for i in range(len(images1)):
    image1 = File(os.path.join(cwd,'Kprojdir/' + images1[i]))
    image2 = File(os.path.join(cwd,'Kprojdir/' + images2[i]))
    output_file = File(os.path.join(cwd,'diffdir/' + outputs[i]))
    template = File(os.path.join(cwd,'Ktemplate.hdr'))
    
    outputs2.append(mDiff_parsl(inputs=[image1, image2, output_file, template]))
    
outputs2 = [i.result() for i in outputs2]

# Alternative DiffExec command

#montage.mDiffExec(File(os.path.join(cwd,'diffs.tbl')), 
                 # File(os.path.join(cwd,'Ktemplate.hdr')), 
               #   os.path.join(cwd,'diffdir/'),
                #  proj_dir=os.path.join(cwd,'Kprojdir/'))

In [60]:
output_1 = montage.mFitplane(os.path.join(cwd,'diffdir/' + outputs[0]))

In [61]:
print(output_1)

stat : OK
a : -0.00469527
b : -0.00454195
c : 6.49254
crpix1 : -1249.5
crpix2 : -462
xmin : 1250.5
xmax : 1325.5
ymin : 463
ymax : 1284
xcenter : 1288.1
ycenter : 872.587
npixel : 45203
rms : 1.40346
boxx : 1288
boxy : 873.5
boxwidth : 823
boxheight : 77
boxang : 90


In [62]:
montage.mFitExec(File(os.path.join(cwd,'diffs.tbl')), 
                 File(os.path.join(cwd,'fits.tbl')), 
                 os.path.join(cwd,'diffdir/'))

stat : OK
count : 233
failed : 0
missing : 0

In [63]:
montage.mBgModel(File(os.path.join(cwd,'images.tbl')), 
                 File(os.path.join(cwd,'fits.tbl')), 
                 File(os.path.join(cwd,'corrections.tbl')))
os.mkdir(os.path.join(cwd,'corrdir'))

stat : OK

In [107]:
corrections = pd.read_csv('corrections.tbl', comment='|', delim_whitespace=True)
corrections.loc[90] = list(corrections.columns)
corrections.columns = ['id','a','b','c']

In [117]:
montage.mBgExec(File(os.path.join(cwd,'images.tbl')), 
                File(os.path.join(cwd,'corrections.tbl')), 
                os.path.join(cwd,'corrdir'), 
                proj_dir=os.path.join(cwd,'Kprojdir'))

stat : OK
count : 91
nocorrection : 0
failed : 0

In [149]:
with io.capture_output() as captured:
    montage.mAdd(File(os.path.join(cwd,'images.tbl')), 
             File(os.path.join(cwd,'Ktemplate.hdr')), 
             File(os.path.join(cwd,'m17.fits')))
    !mViewer -ct 1 -gray m17.fits -1s max gaussian-log -out m17.png

![](./images/m17.png)